In [2]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
from tqdm import tqdm

C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [3]:
# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)


In [4]:
# Check if GPU is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [5]:
# Dataset parameters
img_height, img_width = 224, 224
batch_size = 32
num_classes = 4  # Real, Altered-Easy, Altered-Medium, Altered-Hard
num_epochs = 30

In [6]:
# Define directories
base_dir = "E:/Biometrics/Dataset"  # Replace with your dataset path
if not os.path.exists(base_dir):
    print(f"Error: Directory '{base_dir}' does not exist")
    exit(1)
else:
    print("Dir read")
train_dir = os.path.join(base_dir, 'Train')
val_dir = os.path.join(base_dir, 'Val')
test_dir = os.path.join(base_dir, 'Test')

Dir read


In [7]:
# Data transformations
# For training - with augmentations specific to fingerprints
train_transforms = transforms.Compose([
    transforms.Resize((img_height, img_width)),
    transforms.RandomRotation(20),
    transforms.RandomResizedCrop(size=(img_height, img_width), scale=(0.9, 1.0)),
    transforms.ColorJitter(brightness=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [8]:
# For validation and testing - just resize and normalize
val_test_transforms = transforms.Compose([
    transforms.Resize((img_height, img_width)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [9]:
# Load datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transforms)
val_dataset = datasets.ImageFolder(root=val_dir, transform=val_test_transforms)
test_dataset = datasets.ImageFolder(root=test_dir, transform=val_test_transforms)

In [10]:
# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)


In [11]:
# Print class mapping
class_names = train_dataset.classes
print(f"Class mapping: {train_dataset.class_to_idx}")
print(f"Classes: {class_names}")

Class mapping: {'Altered-Easy': 0, 'Altered-Hard': 1, 'Altered-Medium': 2, 'Real': 3}
Classes: ['Altered-Easy', 'Altered-Hard', 'Altered-Medium', 'Real']


In [12]:

# Load pre-trained EfficientNet model
model = efficientnet_b0(weights=EfficientNet_B0_Weights.IMAGENET1K_V1)

In [13]:
# Freeze base model parameters
for param in model.features.parameters():
    param.requires_grad = False

In [14]:
# Modify the classifier
num_ftrs = model.classifier[1].in_features
model.classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(num_ftrs, num_classes)
)


In [15]:
# Move model to device
model = model.to(device)


In [16]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)

C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [17]:
# Training function
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    best_val_acc = 0.0
    best_model_wts = None
    
    # History for plotting
    history = {
        'train_loss': [], 'train_acc': [],
        'val_loss': [], 'val_acc': []
    }
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)
        
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
                dataloader = train_loader
            else:
                model.eval()
                dataloader = val_loader
            
            running_loss = 0.0
            running_corrects = 0
            
            # Iterate over data
            for inputs, labels in tqdm(dataloader, desc=f"{phase}"):
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                # Zero the parameter gradients
                optimizer.zero_grad()
                
                # Forward pass
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    
                    # Backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                
                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            
            epoch_loss = running_loss / len(dataloader.dataset)
            epoch_acc = running_corrects.double() / len(dataloader.dataset)
            
            # Update history
            if phase == 'train':
                history['train_loss'].append(epoch_loss)
                history['train_acc'].append(epoch_acc.cpu().item())
            else:
                history['val_loss'].append(epoch_loss)
                history['val_acc'].append(epoch_acc.cpu().item())
                # Update LR scheduler
                scheduler.step(epoch_loss)
                
            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            
            # Save best model
            if phase == 'val' and epoch_acc > best_val_acc:
                best_val_acc = epoch_acc
                best_model_wts = model.state_dict().copy()
                torch.save(model.state_dict(), 'best_fingerprint_efficientnet.pth')
                print("Saved new best model")
        
        print()
    
    # After all epochs, load best model weights
    model.load_state_dict(best_model_wts)
    return model, history

In [18]:
# Phase 1: Train only the classifier
print("Phase 1: Training classifier only")
model_phase1, history_phase1 = train_model(model, criterion, optimizer, scheduler, num_epochs=10)

Phase 1: Training classifier only
Epoch 1/10
----------


train: 100%|██████████| 525/525 [17:09<00:00,  1.96s/it]


train Loss: 1.0370 Acc: 0.5387


val: 100%|██████████| 75/75 [02:54<00:00,  2.32s/it] 


val Loss: 0.9000 Acc: 0.6050
Saved new best model

Epoch 2/10
----------


...

val Loss: 0.8294 Acc: 0.6333

Epoch 10/10
----------


train: 100%|██████████| 525/525 [12:58<00:00,  1.48s/it]


train Loss: 0.8811 Acc: 0.6058


val: 100%|██████████| 75/75 [01:50<00:00,  1.47s/it]


val Loss: 0.8422 Acc: 0.6312



In [19]:
# Phase 2: Fine-tune the last few layers of EfficientNet
print("Phase 2: Fine-tuning last layers of EfficientNet")

Phase 2: Fine-tuning last layers of EfficientNet


In [20]:
# Unfreeze the last few layers
for i, param in enumerate(model.features.parameters()):
    if i > len(list(model.features.parameters())) - 20:  # Unfreeze last 20 layers
        param.requires_grad = True

In [21]:
# Lower learning rate for fine-tuning
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)

In [ ]:
# Continue training
model_phase2, history_phase2 = train_model(model, criterion, optimizer, scheduler, num_epochs=20)

Epoch 1/20
----------


train: 100%|██████████| 525/525 [13:45<00:00,  1.57s/it]


train Loss: 0.7184 Acc: 0.6795


val: 100%|██████████| 75/75 [01:38<00:00,  1.32s/it]


val Loss: 0.5853 Acc: 0.7308
Saved new best model

Epoch 2/20
----------


...

train: 100%|██████████| 525/525 [16:40<00:00,  1.91s/it]


train Loss: 0.4100 Acc: 0.8106


val: 100%|██████████| 75/75 [02:57<00:00,  2.37s/it]


val Loss: 0.3726 Acc: 0.8229
Saved new best model

Epoch 9/20
----------


train:   0%|          | 0/525 [00:00<?, ?it/s]

In [ ]:
# Save final model
torch.save(model.state_dict(), 'final_fingerprint_efficientnet.pth')
print("Final model saved!")

In [ ]:
# Evaluate on test set
def evaluate_model(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []
    test_loss = 0.0
    correct = 0
    
    with torch.no_grad():
        for inputs, labels in tqdm(test_loader, desc="Testing"):
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            correct += torch.sum(preds == labels.data)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    test_loss = test_loss / len(test_loader.dataset)
    test_acc = correct.double() / len(test_loader.dataset)
    
    print(f"Test Loss: {test_loss:.4f}")
    print(f"Test Accuracy: {test_acc:.4f}")
    
    # Compute confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    
    # Print classification report
    print("\nClassification Report:")
    print(classification_report(all_labels, all_preds, target_names=class_names))
    
    return all_preds, all_labels, cm

In [ ]:
# Evaluate the final model
y_pred, y_true, cm = evaluate_model(model, test_loader)

In [ ]:
# Plot confusion matrix
plt.figure(figsize=(10, 8))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names, rotation=45)
plt.yticks(tick_marks, class_names)

plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.savefig('confusion_matrix.png')


In [ ]:
# Function to plot training history
def plot_training_history(history_phase1, history_phase2=None):
    plt.figure(figsize=(12, 4))
    
    # Plot accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history_phase1['train_acc'], label='Phase 1 Train')
    plt.plot(history_phase1['val_acc'], label='Phase 1 Validation')
    
    if history_phase2 is not None:
        start_epoch = len(history_phase1['train_acc'])
        epochs_range = range(start_epoch, start_epoch + len(history_phase2['train_acc']))
        plt.plot(epochs_range, history_phase2['train_acc'], label='Phase 2 Train')
        plt.plot(epochs_range, history_phase2['val_acc'], label='Phase 2 Validation')
    
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(loc='lower right')
    
    # Plot loss
    plt.subplot(1, 2, 2)
    plt.plot(history_phase1['train_loss'], label='Phase 1 Train')
    plt.plot(history_phase1['val_loss'], label='Phase 1 Validation')
    
    if history_phase2 is not None:
        plt.plot(epochs_range, history_phase2['train_loss'], label='Phase 2 Train')
        plt.plot(epochs_range, history_phase2['val_loss'], label='Phase 2 Validation')
    
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(loc='upper right')
    
    plt.tight_layout()
    plt.savefig('training_history.png')
    plt.show()

In [ ]:
# Plot training history
history_phase1_dict = {
    'train_loss': history_phase1['train_loss'],
    'val_loss': history_phase1['val_loss'],
    'train_acc': history_phase1['train_acc'],
    'val_acc': history_phase1['val_acc']
}

history_phase2_dict = {
    'train_loss': history_phase2['train_loss'],
    'val_loss': history_phase2['val_loss'],
    'train_acc': history_phase2['train_acc'],
    'val_acc': history_phase2['val_acc']
}

plot_training_history(history_phase1_dict, history_phase2_dict)